In [1]:
import numpy as np
import cv2
import keras
from keras.applications.inception_v3 import InceptionV3, preprocess_input

Using TensorFlow backend.


In [2]:
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        new_image = cv2.resize(image, new_shape)
        images_list.append(new_image)
    return np.asarray(images_list)

In [3]:
def calculate_inception_score(images, n_split=10, eps=1E-16):
    model = InceptionV3()
    scores = list()
    n_part = np.floor(images.shape[0] / n_split)
    for i in range(n_split):
        print(f"Run: {i+1}")
        ix_start, ix_end = int(i * n_part), int((i+1) * n_part)
        subset = images[ix_start:ix_end]
        subset = subset.astype('float32')
        subset = scale_images(subset, (299,299))
        subset = preprocess_input(subset)
        p_yx = model.predict(subset)
        p_y = np.expand_dims(p_yx.mean(axis=0), 0)
        kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
        sum_kl_d = kl_d.sum(axis=1)
        avg_kl_d = np.mean(sum_kl_d)
        is_score = np.exp(avg_kl_d)
        scores.append(is_score)
        
    is_avg, is_std = np.mean(scores), np.std(scores)
    return is_avg, is_std

In [4]:
(images, _), (_, _) = keras.datasets.cifar10.load_data()
np.random.shuffle(images)
print('loaded', images.shape)
is_avg, is_std = calculate_inception_score(images)
print('score', is_avg, is_std)

loaded (50000, 32, 32, 3)
Run: 1
Run: 2
Run: 3
Run: 4
Run: 5
Run: 6
Run: 7
Run: 8
Run: 9
Run: 10
score 10.792273 0.19545591
